In [87]:
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

In [88]:
data_label0 = np.load("/content/final_data_truelabel0.npy", allow_pickle=True)
data_label1 = np.load("/content/final_data_label1.npy", allow_pickle=True)

In [89]:
data_label0 = data_label0[:27]

In [90]:
y = len(data_label0) * [0] + len(data_label1) * [1]

In [91]:
y = np.array(y)

In [92]:
data_label0.shape

(27, 75, 6, 2)

In [93]:
data_label1.shape

(27, 75, 6, 2)

In [94]:
X = np.concatenate((data_label0, data_label1))

In [95]:
X.shape, y.shape

((54, 75, 6, 2), (54,))

In [96]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=21)

In [97]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((48, 75, 6, 2), (48,), (6, 75, 6, 2), (6,))

In [98]:
X_train = X_train.reshape((-1, 75, 12))
X_test  = X_test.reshape((-1, 75, 12))

In [99]:
import tensorflow as tf

In [100]:
tf.random.set_seed(21)

In [112]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(75,12)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [113]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_18 (LSTM)               (None, 75, 64)            19712     
_________________________________________________________________
lstm_19 (LSTM)               (None, 75, 128)           98816     
_________________________________________________________________
lstm_20 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_18 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_19 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 33        
Total params: 174,209
Trainable params: 174,209
Non-trainable params: 0
________________________________________________

In [114]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

In [115]:
# from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint

In [116]:
# es = EarlyStopping(monitor="accuracy", mode="max", patience=30)
chkpt = ModelCheckpoint("/content/checkpoints", monitor="loss", save_best_only=True, mode="min")

In [117]:
model.fit(x=X_train, y=y_train, epochs=75, callbacks=[chkpt])

Epoch 1/75
2/2 [==============================] - 3s 329ms/step - loss: 0.6923 - accuracy: 0.5000
INFO:tensorflow:Assets written to: /content/checkpoints/assets
Epoch 2/75
2/2 [==============================] - 1s 344ms/step - loss: 0.6914 - accuracy: 0.8958
INFO:tensorflow:Assets written to: /content/checkpoints/assets
Epoch 3/75
2/2 [==============================] - 1s 323ms/step - loss: 0.6896 - accuracy: 0.5000
INFO:tensorflow:Assets written to: /content/checkpoints/assets
Epoch 4/75
2/2 [==============================] - 1s 320ms/step - loss: 0.6864 - accuracy: 0.5000
INFO:tensorflow:Assets written to: /content/checkpoints/assets
Epoch 5/75
2/2 [==============================] - 1s 349ms/step - loss: 0.6813 - accuracy: 0.5000
INFO:tensorflow:Assets written to: /content/checkpoints/assets
Epoch 6/75
2/2 [==============================] - 1s 324ms/step - loss: 0.6685 - accuracy: 0.5000
INFO:tensorflow:Assets written to: /content/checkpoints/assets
Epoch 7/75
2/2 [==================

In [118]:
!ls checkpoints

assets	keras_metadata.pb  saved_model.pb  variables


In [120]:
model.load_weights("/content/checkpoints")

In [121]:
y_test

array([1, 0, 0, 1, 1, 0])

In [122]:
model.predict(X_test)

array([[1.        ],
       [0.47899243],
       [0.47961256],
       [1.        ],
       [1.        ],
       [0.47965556]], dtype=float32)

In [123]:
model.save('final_model.h5')